# MGMTMSA 403: Optimization
# Assignment 4: Logistic Regression and Gradient Descent
### Elen Sahakyan (UID: 706310696)
### Jane Lee (UID: )

In [4]:
# Import libraries
import numpy as np
from numpy import genfromtxt
import pandas as pd
from gurobipy import *

**Question 1:** Train a logistic regression classifier using gradient descent on the training data set. <br> Starting with a step size of `0.00001`, and `2,000` iterations, experiment with different step sizes and termination criteria to obtain a good model fit on the testing data set.

In [31]:
# Load data
data = genfromtxt('LRTrain.csv', delimiter = ',', skip_header = 1)
data.shape

(300, 31)

In [6]:
n = data.shape[0]
d = data.shape[1]-1

In [36]:
T = 2100  # number of iterations
eps = 1e-8  # tolerance
step = 0.00004  # step size

# get feature data
x = data[:, 0:d]

# get label data
y = data[:, d]

w = np.zeros(d)

## Define gradient function
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def grad(w, x, y):
    # initialize d-dimensional gradient vector
    g = np.zeros(d)

    # construct gradient vector by looping through each of the n observations to sum them
    for i in range(n):
        z = np.dot(w, x[i])
        prediction = sigmoid(z)
        g = g + (prediction - y[i]) * x[i]

    return (1 / n) * g

## Define negative log likelihood function
def fval(w, x, y):
    v = 0
    for i in range(n):
        z = np.dot(w, x[i])
        v = v + y[i] * np.log(1 + np.exp(-np.clip(z, -500, 500))) + (1 - y[i]) * np.log(1 + np.exp(np.clip(z, -500, 500)))
    return -(1 / n) * v

## Define norm of the gradient
def gradnorm(w, x, y):
    return np.linalg.norm(grad(w, x, y))

## Perform gradient descent
for t in range(T):
    # update weights
    w = w - step * grad(w, x, y)

    # print output for each iteration
    print("Step count: " + str(t) + ", Negative log likelihood: " + str(fval(w, x, y)) +
          ", gradient norm: " + str(gradnorm(w, x, y)))

    # Check termination criterion
    if gradnorm(w, x, y) < eps:
        break

# save final weight vector
w_hat = w

Step count: 0, Negative log likelihood: -1.6563829152486882, gradient norm: 418.88178854819887
Step count: 1, Negative log likelihood: -8.27251677854852, gradient norm: 634.0451589002729
Step count: 2, Negative log likelihood: -5.453245906718302, gradient norm: 455.43722272954943
Step count: 3, Negative log likelihood: -3.6883429040849114, gradient norm: 627.4836539278753
Step count: 4, Negative log likelihood: -8.523311168922026, gradient norm: 455.8436871449809
Step count: 5, Negative log likelihood: -0.6255963053333304, gradient norm: 181.7025680009113
Step count: 6, Negative log likelihood: -3.833502307684991, gradient norm: 629.3166410361287
Step count: 7, Negative log likelihood: -8.308599435533251, gradient norm: 455.83604892261144
Step count: 8, Negative log likelihood: -0.540183431314659, gradient norm: 109.60306577948846
Step count: 9, Negative log likelihood: -2.137453627102345, gradient norm: 610.4458481696006
Step count: 10, Negative log likelihood: -9.066131446149429, gra

To understand the performance of the model, we monitor how the objective function, i.e. the negative log likelihood and the gradient norm change in response to the model parameters.
 
- The step size (step) controls how quickly the gradient norm is changing. If the change is gradual and slow, we can increase the step size, and vice versa for the opposite scenario.  
- Tolerance (eps) controls the convergence, which can be observed by looking at the negative log likelihood and gradient norm changes. If the changes are minimal, it indicates that the model may have converged, and it will require increasing the tolerance level. 
- Number of iterations (T), is the time that the model is training. Too much time, or too high number of iterations for training the model increases the risk of overfitting.  

After experimenting with the parameters, we can observe that a number of iterations of `2,100`, a tolerance of `1e-8`, and step size of `0.00004` produces the model with the best model fit in terms of convergence, overfitting and change in step size. The results are further validated with accuracy metrics on the testing dataset. 

In [43]:
print(w_hat)

[-1.57037787e-02 -1.81146422e-02 -8.44823372e-02 -2.48757591e-02
 -1.00453240e-04  2.58724634e-04  5.46966021e-04  2.23301889e-04
 -2.22400259e-04 -9.86974377e-05 -7.53439422e-05 -1.43701794e-03
  1.58819520e-03  3.24104699e-02  3.07399779e-06  7.39210422e-05
  1.10055289e-04  2.69033388e-05 -7.58554634e-06  1.70637472e-06
 -1.66692944e-02 -2.28277108e-02 -7.55792465e-02  3.94856567e-02
 -1.06452178e-04  8.98511553e-04  1.29251810e-03  3.41085199e-04
 -2.63350976e-04 -6.01513716e-05]


**Question 2:** After training a logistic regression classifier, compute the true positive rate (TPR), false positive rate (FPR), true negative rate (TNR), and false negative rate (FNR) on the test dataset for different thresholds. 

In [44]:
# Load test data
data = genfromtxt('LRTest.csv', delimiter=',', skip_header=1)

n = data.shape[0]
d = data.shape[1] - 1

# get feature data
x = data[:, 0:d]

# get label data
y = data[:, d]

# set threshold
t = np.arange(0, 1.1, 0.1)

# Initialize lists to store results
thresholds_list = []
TPR_list = []
FPR_list = []
TNR_list = []
FNR_list = []

# make predictions on test data using trained weights w_hat
for threshold in t:
    pred = np.zeros(n)
    for i in range(n):
        if 1 / (1 + np.exp(-np.dot(w_hat, x[i, :]))) > threshold:
            pred[i] = 1
        elif 1 / (1 + np.exp(-np.dot(w_hat, x[i, :]))) < threshold:
            pred[i] = 0

    TP = np.sum((pred == 1) & (y == 1))  # number of true positives
    FP = np.sum((pred == 1) & (y == 0))  # number of false positives
    TN = np.sum((pred == 0) & (y == 0))  # number of true negatives
    FN = np.sum((pred == 0) & (y == 1))  # number of false negatives

    # Calculate TPR, FPR, TNR, and FNR
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    TNR = TN / (FP + TN)
    FNR = FN / (TP + FN)

    # Append results to lists
    thresholds_list.append(threshold)
    TPR_list.append(TPR)
    FPR_list.append(FPR)
    TNR_list.append(TNR)
    FNR_list.append(FNR)

# Create a DataFrame
results_df = pd.DataFrame({
    't': thresholds_list,
    'TPR': TPR_list,
    'FPR': FPR_list,
    'TNR': TNR_list,
    'FNR': FNR_list
})

# Display the DataFrame
print(f'   Accuracy Metrics for Different Thresholds\n')
results_df

   Accuracy Metrics for Different Thresholds



,t,TPR,FPR,TNR,FNR
0,0.0,1.000000,1.000000,0.000000,0.000000
1,0.1,0.969388,0.228070,0.771930,0.030612
2,0.2,0.918367,0.128655,0.871345,0.081633
3,0.3,0.897959,0.099415,0.900585,0.102041
4,0.4,0.877551,0.052632,0.947368,0.122449
5,0.5,0.877551,0.046784,0.953216,0.122449
6,0.6,0.867347,0.040936,0.959064,0.132653
7,0.7,0.857143,0.029240,0.970760,0.142857
8,0.8,0.826531,0.029240,0.970760,0.173469
9,0.9,0.785714,0.011696,0.988304,0.214286



In the context of tumor identification, both the True Positive Rate (TPR) and False Positive Rate (FPR) are crucial. However, the prevailing practice in the industry leans towards minimizing FPR. This preference arises from the desire to reduce instances where patients and healthy cells undergo unnecessary treatment for a tumor.

Extreme threshold values tend to bias towards one of the metrics. The middle thresholds, 0.6 and 0.7, offer the optimal balance between TPR and FPR.